In [ ]:
# --- BI-Net exploration in a Jupyter Notebook ---

import torch
import sys
import os

PROJECT_ROOT = os.path.abspath("..")  # points to my_project
sys.path.append(PROJECT_ROOT)

# Double-check
print("PROJECT_ROOT =", PROJECT_ROOT)
print("Contents:", os.listdir(PROJECT_ROOT))

# Now import the function
from src.models.bi_net import BiNet

# Import dataset class
from src.dataset.pc_dataset import PointCloudDataset

In [2]:
##############################################
# 2) Define Model Hyperparameters
##############################################

# Example settings:
batch_size = 1        # how many point clouds in a batch
latent_dim = 96       # dimension of the latent code
features_g = [latent_dim, 128, 128, 64, 64, 32, 3]  # for decoder/generator layers
degrees    = [4, 4, 4, 4, 4, 2]   # expansion degrees in the TreeGCN
enc_disc_feat = [3, 64, 128, 256, 512] # for encoder/discriminator
support = 10          # support factor in the TreeGCN

In [ ]:
# Create the BI-Net
binet = BiNet(
    batch_size=batch_size,
    features_g=features_g,
    degrees=degrees,
    enc_disc_feat=enc_disc_feat,
    latent_dim=latent_dim,
    support=support
)

# Move to GPU if available:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
binet.to(device)
binet.eval()

print("BI-Net model created and set to eval mode.\n")

In [ ]:
##############################################
# 3) Forward Pass (Auto-Encoder direction)
##############################################
# Feed random data shaped (batch_size, N_points, 3)
dummy_input = torch.randn(batch_size, 512, 3).to(device)  # e.g. 512 points per cloud
latent_code = binet.encode(dummy_input)   # Encode
print("Encoded latent shape:", latent_code.shape)

rec_points = binet.decode(latent_code)    # Decode
print("Reconstructed points shape:", rec_points.shape)

In [ ]:
##############################################
# 4) Reverse Pass (GAN direction)
##############################################
# Generate from random noise
z = torch.randn(batch_size, latent_dim).to(device)
fake_points = binet.generate(z)
print("Generated (fake) points shape:", fake_points.shape)

# Discriminate real vs. fake
disc_real = binet.discriminate(dummy_input)
disc_fake = binet.discriminate(fake_points)
print("Discriminator score for real shape:", disc_real.shape)
print("Discriminator score for fake shape:", disc_fake.shape)

In [ ]:
##############################################
# 5) Test on Another Random Point Cloud
##############################################
# Suppose we have a new random input shaped (1, 1024, 3) simulating an external "random" point cloud
random_pc = torch.randn(1, 1024, 3).to(device)
print("\nTesting BI-Net on another random point cloud of shape:", random_pc.shape)

# AE pass on the second random cloud
latent_code_2 = binet.encode(random_pc)
rec_points_2 = binet.decode(latent_code_2)

print("Encoded shape (2nd random input):", latent_code_2.shape)
print("Reconstructed shape (2nd random input):", rec_points_2.shape)

# Optionally, discriminate it as "real"
disc_real_2 = binet.discriminate(random_pc)
print("Discriminator output on 2nd random input:", disc_real_2.shape)

In [ ]:
##############################################
# 6) Final Summary
##############################################
print("\n--- BI-Net Exploration Summary ---")
print("AE direction => Encoded shape (1st input):", latent_code.shape, 
      "| Reconstructed shape (1st input):", rec_points.shape)
print("GAN direction => Fake shape:", fake_points.shape,
      "| Disc score shapes (real, fake):", disc_real.shape, disc_fake.shape)
print("2nd random input => Encoded shape:", latent_code_2.shape, 
      "| Reconstructed shape:", rec_points_2.shape,
      "| Disc output (as real):", disc_real_2.shape)
print("Everything is running. Customize or extend for training, debugging, or actual data loading.")

In [ ]:
##########################################
# 1) Create the dataset object
##########################################
data_root = os.path.join(PROJECT_ROOT, "data")
train_root = os.path.join(PROJECT_ROOT, "data\\splits\\train")


dataset = PointCloudDataset(root=data_root, split="processed", transform=None)

print(f"Loaded dataset with {len(dataset)} samples (train split).")

In [ ]:
import random
##########################################
# 7) Pick a random index & load one point cloud data/npz
##########################################
if len(dataset) == 0:
    print("Dataset is empty! Please check your dataset folder.")
else:
    rand_idx = random.randint(0, len(dataset) - 1)
    pc_data = dataset[rand_idx]  # shape => (N, 3) Torch Tensor
    print(f"Random index: {rand_idx}, pc_data shape: {pc_data.shape}")

    # Add batch dimension => (1, N, 3)
    pc_data = pc_data.unsqueeze(0)

In [ ]:
##########################################
# 4) Send point cloud to device & run AE pass
##########################################
pc_data = pc_data.to(device)  # shape => (1, N, 3)
with torch.no_grad():
    latent = binet.encode(pc_data)            # shape => (1, latent_dim)
    rec_points = binet.decode(latent)         # shape => (1, N, 3)

print(f"Latent code shape: {latent.shape}")
print(f"Reconstructed points shape: {rec_points.shape}")

In [ ]:
##########################################
# 5) Discriminator on that single real cloud
##########################################
disc_output = binet.discriminate(pc_data)     # shape => (1, 1)
print(f"Discriminator output on real cloud: {disc_output}")

##########################################
# 6) Interpret the Results
##########################################
print("\n--- Single Point Cloud Exploration ---")
print(f"Random index: {rand_idx}, Original shape: {pc_data.shape}")
print(f"Latent code shape: {latent.shape}, Reconstructed shape: {rec_points.shape}")
print(f"Discriminator output on real: {disc_output.shape} => {disc_output}")
print("Done. If your model is untrained, reconstruction may be random. If trained, it should reflect learned geometry.")

In [ ]:
import open3d as o3d
# rec_points: shape => (1, N, 3), e.g. (1, 2048, 3)
rec_np = rec_points[0].cpu().numpy()  # => shape (2048, 3)

# Convert rec_np to Open3D point cloud
pcd_recon = o3d.geometry.PointCloud()
pcd_recon.points = o3d.utility.Vector3dVector(rec_np)

# Visualize
o3d.visualization.draw_geometries([pcd_recon])